<a href="https://colab.research.google.com/github/luiseduaardo/PneuModel/blob/main/notebooks/inferencia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!git clone https://github.com/luiseduaardo/PneuModel.git
%cd PneuModel

Cloning into 'PneuModel'...
remote: Enumerating objects: 5960, done.
remote: Counting objects: 100% (59/59), done.
remote: Compressing objects: 100% (56/56), done.
remote: Total 5960 (delta 28), reused 8 (delta 3), pack-reused 5901 (from 3)
Receiving objects: 100% (5960/5960), 1.26 GiB | 32.12 MiB/s, done.
Resolving deltas: 100% (46/46), done.
/content/PneuModel


In [3]:
!pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.9 MB/s eta 0:00:00


In [4]:
import os
import zipfile

FILE_ID = '1YLeauT4Kwl6EpdpcUesU6WdE5cA12Y4K'
ZIP_NAME = 'ligia-compviz.zip'
EXTRACT_PATH = './dataset'

def download_and_extract():
    if os.path.exists(ZIP_NAME):
        os.remove(ZIP_NAME)

    command = f"""
      FILEID="{FILE_ID}";
      FILENAME="{ZIP_NAME}";
      wget --quiet --save-cookies cookies.txt --keep-session-cookies \
      "https://drive.google.com/uc?export=download&id=$FILEID" -O response.html;
      UUID=$(grep -o 'name="uuid" value="[^"]*' response.html | sed 's/.*value="//');
      wget --load-cookies cookies.txt \
      "https://drive.usercontent.google.com/download?id=$FILEID&export=download&confirm=t&uuid=$UUID" \
      -O "$FILENAME";
      rm -f cookies.txt response.html;
    """

    os.system(command)

    if os.path.exists(ZIP_NAME) and zipfile.is_zipfile(ZIP_NAME):
        print("Arquivo baixado com sucesso. Extraindo...")
        with zipfile.ZipFile(ZIP_NAME, 'r') as zip_ref:
            zip_ref.extractall(EXTRACT_PATH)
        print(f"✅ Dataset pronto em {EXTRACT_PATH}")

download_and_extract()

Arquivo baixado com sucesso. Extraindo...
✅ Dataset pronto em ./dataset


In [5]:
from google.colab import drive
import tensorflow as tf
from tensorflow.keras.models import load_model
import pandas as pd
import os

In [ ]:
drive.mount('/content/drive')

path_teste = "./dataset/test_images"
model_path_drive = '/content/drive/MyDrive/PneuModel/modelo.keras'

# carregamento do modelo
model = load_model(model_path_drive)
print("✅ Modelo carregado com sucesso!")

Mounted at /content/drive
✅ Modelo carregado com sucesso!


In [10]:
# pré processamento da imagem
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator()

test_generator = test_datagen.flow_from_directory(
    path_teste,
    target_size=(224, 224),
    batch_size=1,
    class_mode=None,
    shuffle=False
)

# realiza a previsão
previsoes = model.predict(test_generator)

Found 624 images belonging to 1 classes.


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


624/624 ━━━━━━━━━━━━━━━━━━━━ 381s 610ms/step


In [ ]:
# mapeamento dos ids para formatação pro kaggle
ids = [os.path.basename(x) for x in test_generator.filenames]

submissao = pd.DataFrame({
    'id': ids,
    'target': previsoes.flatten()
})

submissao.head()

,id,target
0,img_0001.jpeg,0.382500
1,img_0002.jpeg,0.694491
2,img_0003.jpeg,0.999546
3,img_0004.jpeg,0.003125
4,img_0005.jpeg,0.134016


In [12]:
path_final = './submission.csv'
submissao.to_csv(path_final, index=False, sep=',')